In [67]:
%matplotlib qt

import numpy as np
import numpy.ma as ma
import itertools

from sys import getsizeof
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import Axes3D

import IPython.display as ipd

import seaborn as sns
import pandas as pd

In [68]:
def base_to_the_n_array(base, num_bases=8):
    '''
    e.g. 1,2,4,5 for base 2 num_bases=4
    '''
    return np.flip(np.array([base**n for n in range(num_bases)]))

# starters: binary numbers into base-m space
def binary_to_n_polynomial_powers(bin_num, n_powers):
    
    return np.array([binary_to_polynomial_power(bin_num, i+2) for i in range(n_powers)])

def binary_to_polynomial_power(bin_val, power_n):
    return val_base_m_to_polynomial_power_n(bin_val, 2, power_n)

# wider approach (values base)
def val_base_m_to_polynomial_power_n(bin_val, base_m, power_n):
    if (bin_val == 0):
        return 0
    return bits_to_polynomial_power(value_to_array_in_base(bin_val, base_m), power_n)

# # utility for dot product against the 
# def val_in_reverse_base_m(n, base):
#     return np.flip(value_to_array_in_base(n, range(base)))

def value_to_array_in_base(n, base):
    bases = range(base)
    if n == 0: return bases[0]
    b = len(bases)
    digits = []
    while n > 0:
        digits = [bases[n % b]] + digits
        n  = n // b
    return np.array(digits)

def value_to_string_in_base(n, base):
    return np.array2string(value_to_array_in_base(n, base), separator='')[1:-1]

def bits_to_polynomial_power(bits_array, power):
    return np.dot(bits_array, base_to_the_n_array(power, num_bases=np.size(bits_array)))


def coeff_projection_to_base_n(array, base):
    return np.array([binary_to_polynomial_power(n, base) for n in array])

def coeff_base_m_projection_to_power_n(array, base_m, power_n):
    return np.array([val_base_m_to_polynomial_power_n(base_to_the_n_array(n), base_m, power_n) for n in array])

# primeness
def prime_filter(a):
    return np.array([n for n in a if is_prime(n)])

def prime_filter_with_indices(a):
    return np.array([(i, n) for i, n in enumerate(a) if is_prime(n)])

def is_prime(n):
    if (n <= 1) :
        return 0
    if (n <= 3) :
        return 1
    if (n % 2 == 0 or n % 3 == 0) :
        return 0
    i = 5
    while(i * i <= n) :
        if (n % i == 0 or n % (i + 2) == 0):
            return 0
        i = i + 6
    return 1

prime_func = np.vectorize(is_prime)

## a list of quantitities

In [165]:
count_to = 30
integers = np.linspace(1, count_to, count_to, dtype=np.int64)
integers

# from those the primes
primes = prime_filter(integers)
primes

# from which we have the option of choosing either... to investigate
source_list = integers
source_list

source_len = len(source_list)

## represented in a number system: 
### ...binary

In [166]:
binstr = np.array([value_to_string_in_base(quant, 2) for quant in source_list])
binstr

array(['1', '10', '11', '100', '101', '110', '111', '1000', '1001',
       '1010', '1011', '1100', '1101', '1110', '1111', '10000', '10001',
       '10010', '10011', '10100', '10101', '10110', '10111', '11000',
       '11001', '11010', '11011', '11100', '11101', '11110'], dtype='<U5')

### ... or ternary

In [167]:
terstr = np.array([value_to_string_in_base(quant, 3) for quant in source_list])
terstr

array(['1', '2', '10', '11', '12', '20', '21', '22', '100', '101', '102',
       '110', '111', '112', '120', '121', '122', '200', '201', '202',
       '210', '211', '212', '220', '221', '222', '1000', '1001', '1002',
       '1010'], dtype='<U4')

### ... etc.

In [168]:
quastr = np.array([value_to_string_in_base(quant, 4) for quant in source_list])
quastr

array(['1', '2', '3', '10', '11', '12', '13', '20', '21', '22', '23',
       '30', '31', '32', '33', '100', '101', '102', '103', '110', '111',
       '112', '113', '120', '121', '122', '123', '130', '131', '132'],
      dtype='<U3')

## represented as arrays of coefficients of a given base, so that we are not limited by symbols

### ... binary (in numerical order)

In [169]:
bins = np.array([value_to_array_in_base(quant, 2) for quant in source_list])

### dot product by the powers of a base to see the original quantities

In [170]:
bases = [base_to_the_n_array(2, len(bins[i])) for i in range(source_len)]

In [171]:
# dot product of the coefficients and bases for consecutive representations of powers
bins_plane = [[bits_to_polynomial_power(bins[i], j) for i in range(source_len)] for j in range(source_len)]

In [172]:
# fig = plt.figure(figsize=(12,12))
# ax = fig.add_subplot(1, 1, 1)
# ax.set_yscale('log', basey=2)
# ax.set_xticks(primes, minor=False)


# plt.plot(bins_plane[1:], '-o')

# plt.show

In [173]:
rolling_bases_coeffs = [[val_base_m_to_polynomial_power_n(i, 2, n) for i in range(source_len)] for n in range(source_len)]

In [174]:
rolling_bases_coeffs_mask = prime_func(rolling_bases_coeffs)
rolling_bases_coeffs_tower = rolling_bases_coeffs_mask * rolling_bases_coeffs

In [175]:
# fig = plt.figure(figsize=(12,12))
# ax = fig.add_subplot(1, 1, 1)
# ax.set_yscale('log', basey=2)
# ax.set_xticks(primes, minor=False)

# plt.plot(rolling_bases_coeffs[1:], '-')
# plt.plot(rolling_bases_coeffs_tower[1:], '^')

# plt.show

In [176]:
rolling_bases_coeffs = np.array([np.array([np.array([val_base_m_to_polynomial_power_n(i, m+2, n+1) for i in range(source_len)]) for m in range(source_len)]) for n in range(source_len)])

In [177]:
rolling_bases_coeffs_mask = prime_func(rolling_bases_coeffs)
rolling_bases_coeffs_tower = rolling_bases_coeffs_mask * rolling_bases_coeffs

In [186]:
what_to_graph = ma.masked_outside(rolling_bases_coeffs_tower, 11, 23)

a = range(rolling_bases_coeffs.shape[0])
b = range(rolling_bases_coeffs.shape[1])
c = range(rolling_bases_coeffs.shape[2])

locations = np.array(list(itertools.product(a, b, c)))
locs_rows = locations.T

logs = np.log(what_to_graph)
max_val = np.max(logs)
flat = logs.flatten()/max_val

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=count_to*2, azim=-270)

ax.scatter(locs_rows[0], locs_rows[1], locs_rows[2], c=flat, cmap='viridis', marker='o', s=30)

plt.show

/Users/johntimothysummers/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


<function matplotlib.pyplot.show(*args, **kw)>